In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
from dateutil.utils import today
from pandas.tseries.offsets import MonthEnd

data_dir = Path('../data')

## Inflation data

Source of data: [dane.gov.pl](https://dane.gov.pl/pl/dataset/2055,miesieczne-wskazniki-cen-towarow-i-uslug-konsumpcy)
(access on 04.09.2021).

In [ ]:
inflation = pd.read_csv(
    '../data/Miesieczne_wskazniki_cen_towarow_i_uslug_konsumpcyjnych_od_1982_roku (1).csv',
    sep=';',
    decimal=',',
    usecols=['Sposob prezentacji', 'Rok', 'Miesiac', 'Wartosc']
)

# Get only monthly inflation rate
inflation = inflation[inflation['Sposob prezentacji'] == 'Poprzedni miesiąc = 100']

# Parsing date
inflation['date'] = pd.to_datetime(
    inflation['Rok'].astype(str) + '-' + inflation['Miesiac'].astype(str).str.zfill(2),
    format='%Y-%m'
)

# Set date as the last day of month
inflation['date'] = inflation['date'] + MonthEnd(0)

inflation = inflation \
    .sort_values('date') \
    .rename({'Wartosc': 'inflation_rate'}, axis=1) \
    .set_index('date') \
    ['inflation_rate'].squeeze()

inflation = inflation / 100 - 1

inflation.tail()

In [ ]:
px.line(
    (inflation + 1).cumprod(),
    labels=dict(value="%"),
    title='Cumulative inflation in Poland'
)

## Exchange rates

In [ ]:
exchange_rates = pd.read_csv(
    data_dir / 'Dane historyczne dla USD_PLN.csv',
    usecols=['Data', 'Ostatnio'],
    parse_dates=['Data'],
    decimal=','
)

exchange_rates = exchange_rates.rename(columns={
    'Data': 'date',
    'Ostatnio': 'USD2PLN'
})

In [ ]:
import dateparser

exchange_rates['date'] = exchange_rates['date'] \
    .map(lambda d: dateparser.parse(d, languages=['pl'], settings={'DATE_ORDER': 'MYD'}))

exchange_rates['date'] = exchange_rates['date'] + MonthEnd(0)  #TODO or MonthEnd(0)???
exchange_rates = exchange_rates \
    .set_index('date') \
    .squeeze() \
    .sort_index()

exchange_rate_monthly_return_rates = exchange_rates \
    .rolling(window=2) \
    .apply(lambda x: x[1] / x[0] - 1) \
    .dropna()

exchange_rates.tail()

In [ ]:
px.line(
    exchange_rates,
    labels=dict(value="USD price (PLN)"),
    title='Exchange rate USD -> PLN'
)

## Retail treasury bonds

In Polish: detaliczne obligacje skarbowe.

### 10-years bonds

In [ ]:
edo = pd.read_excel(
    data_dir / 'Dane_dotyczace_obligacji_detalicznych.xls',
    sheet_name='EDO',
    parse_dates=['Początek sprzedaży'],
    index_col='Początek sprzedaży',
).rename(columns={
    'Koniec sprzedaży': 'date',
    'Marża': 'margin'
})[['margin']] \
    .squeeze()[1:]

edo.index = edo.index + MonthEnd(0)

# original data contains yearly return rate, here it's turned into monthly return rate
edo = np.power(edo + 1, 1/12) - 1

# return rate consists of margin and inflation rate
edo = edo + np.maximum(0, inflation)

edo.name = 'bonds_10_years'

### 4-years bonds

In [ ]:
coi = pd.read_excel(
    data_dir / 'Dane_dotyczace_obligacji_detalicznych.xls',
    sheet_name='COI',
    parse_dates=['Początek sprzedaży'],
    index_col='Początek sprzedaży',
).rename(columns={
    'Koniec sprzedaży': 'date',
    'Marża': 'margin'
})[['margin']] \
    .squeeze()[1:]

coi.index = coi.index + MonthEnd(0)

# original data contains yearly return rate, here it's turned into monhly return rate
coi = np.power(coi + 1, 1/12) - 1

# return rate consists of margin and inflation rate
coi = coi + np.maximum(0, inflation[coi.index[0]:])

coi.name = 'bonds_4_years'

coi

### "IR" bonds

In [ ]:
ir = pd.read_excel(
    data_dir / 'Dane_dotyczace_obligacji_detalicznych.xls',
    sheet_name='IR',
    parse_dates=['Początek sprzedaży'],
    index_col='Początek sprzedaży',
    skiprows=1
).rename(columns={
    'Unnamed: 17': 'bank_rate'
})[['bank_rate']] \
    .squeeze()[1:]

ir.index = ir.index + MonthEnd(0)

# original data contains yearly return rate, here it's turned into monhly return rate
ir = np.power(ir + 1, 1/12) - 1

ir.name = 'bonds_ir'

ir

Combining bonds

In [ ]:
combined = pd.Series(
    index=pd.date_range('1995-01-01', today(), freq='M'),
    dtype='float64'
)

combined.update(edo)
combined.update(coi[:'2004-10-01'])
combined.update(ir)

# IR bonds data contains only quarterly return rate, so here holes are filled
# with fecent return rate
combined = combined.ffill()

combined.name = 'bonds_approximated'

## Gold

In [ ]:
gold_prices_monthly = pd.read_excel(
    data_dir / 'Prices.xlsx',
    sheet_name='Monthly_Full',
    skiprows=8,
    index_col='Name',
    usecols=['Name', 'USD'],
    squeeze=True
).rename('ounce_in_USD')

gold_prices_monthly = gold_prices_monthly.dropna()
gold_prices_monthly.index = gold_prices_monthly.index + MonthEnd(0)

In [ ]:
px.line(gold_prices_monthly)

In [ ]:
monthly_return_rates = gold_prices_monthly \
    .rolling(window=2) \
    .apply(lambda x: x[1] / x[0] - 1) \
    .dropna()

gold_monthly_return_rates_in_PLN = (monthly_return_rates + 1) * (exchange_rate_monthly_return_rates + 1) - 1
gold_monthly_return_rates_in_PLN.name = 'gold_PLN'

px.line(monthly_return_rates)

### MSCI World

In [ ]:
from pandas.tseries.offsets import MonthEnd

stock_index = pd.read_excel(
    data_dir / 'historyIndex (3).xls',
    skiprows=6,
    nrows=614,
    parse_dates=['Date'],
    index_col='Date',
    thousands=','
).rename(columns={
    'WORLD Standard (Large+Mid Cap)': 'MSCI_WORLD_USD',
    'EM (EMERGING MARKETS) Standard (Large+Mid Cap)': 'MSCI_EM_USD'
})

stock_index.index = stock_index.index + MonthEnd(0)

In [ ]:
px.line(stock_index, y='MSCI_WORLD_USD')

In [ ]:
msci_em_monthly_return_rates = stock_index['MSCI_EM_USD'] \
    .rolling(window=2) \
    .apply(lambda x: x[1] / x[0] - 1) \
    .dropna()

msci_em_monthly_return_rates_in_PLN = (msci_em_monthly_return_rates + 1) * (exchange_rate_monthly_return_rates + 1) - 1
msci_em_monthly_return_rates_in_PLN.name = 'msci_em_PLN'

msci_world_monthly_return_rates = stock_index['MSCI_WORLD_USD'] \
    .rolling(window=2) \
    .apply(lambda x: x[1] / x[0] - 1) \
    .dropna()

msci_world_monthly_return_rates_in_PLN = (msci_world_monthly_return_rates + 1) * (exchange_rate_monthly_return_rates + 1) - 1
msci_world_monthly_return_rates_in_PLN.name = 'msci_world_PLN'

aaa = msci_world_monthly_return_rates_in_PLN.copy()
aaa.update(msci_em_monthly_return_rates_in_PLN)
msci_em_monthly_return_rates_in_PLN = aaa
msci_em_monthly_return_rates_in_PLN.name = 'msci_em_PLN'

In [ ]:
df = pd.concat(
    [
        inflation,
        gold_monthly_return_rates_in_PLN,
        msci_em_monthly_return_rates_in_PLN,
        msci_world_monthly_return_rates_in_PLN,
        combined
    ],
    axis='columns'
)
df = df.fillna(0)
df

In [ ]:
from typing import List, Optional
from dataclasses import dataclass, asdict
from utils import run_simulation

df = df['1995-01-01':'2020-01-01']

@dataclass
class Config:
    name: str
    distribution: List
    rebalancing: Optional[str] = 'yearly'

configs = [
    Config(
        'only_bonds',
        [('bonds_approximated', 'bonds', 1.0)        ]
    ),
    Config(
        'inflation',
        [('inflation_rate', 'asd', 1.0)]
    ),
    Config(
        'only_gold',
        [('gold_PLN', 'ounces', 1.0)]
    ),
    Config(
        'only_msci_em',
        [('msci_em_PLN', 'ounces', 1.0)]

    ),
    Config (
        'only_msci_world',
        [('msci_world_PLN', 'ounces', 1.0)]
    ),
    Config(
        'strategy',
        [
            ('msci_world_PLN', '1', 0.15),
            ('msci_em_PLN', '2', 0.2),
            ('gold_PLN', '3', 0.15),
            ('bonds_approximated', '4', 0.5)
        ],
        'yearly'
    ),
    Config(
        'strategy2',
        [
            ('msci_world_PLN', '1', 0.15),
            ('msci_em_PLN', '2', 0.2),
            ('gold_PLN', '3', 0.15),
            ('bonds_approximated', '4', 0.5)
        ],
        None
    )
]

for config in configs:
    df[config.name] = run_simulation(
        df,
        distribution=config.distribution,
        initial_investment_value=100,
        rebalancing=config.rebalancing
    )

In [ ]:
px.line(
    df,
    y=[c.name for c in configs]
)

In [ ]:
from utils import calculate_maximum_drawdown, calculate_yearly_return_rate

pd.DataFrame(
    [
        list(calculate_maximum_drawdown(df[c.name])) + [calculate_yearly_return_rate(df[c.name])]
        for c in configs
    ],
    columns=[
        'max_drawdown',
        'max_drawdown_from_time',
        'max_drawdown_till_time',
        'yearly_return_rate'
    ],
    index=[c.name for c in configs]
)